# ASR
本note计划介绍三个模块，第一模块是分步（分为声学模型（AM, acoustic model）和语言模型（LM, language model）），第二模块是end2end部分，第三个模块是目前可用的包和API

## 1. AM&LM
目前主流的声音识别模型是分为两个模块，分别是声学模型和语言模型。声学模型可以理解为对发声的建模，即把输入的语音信号转换为声学表示的输出，**准确的说是给出语音属于某一个声学符号的概率**；而语言模型的作用可以简单的理解为消解多音字的问题，在声学模型给出声学符号（发音序列）后，从候选的文字序列中找出概率最大的字符串序列。   
基于声学模型和语言模型的连续语音识别框图如下所示，
    ![](../../pics/speech_flow.png)

整体语音识别可以表示为如下公式，
$$
\begin{aligned}
W^{*} &= \arg \max_w P(W|Y)\ &(1) \\ 
      &= \arg \max_w \frac{P(Y|W)P(W)}{P(Y)}\ &(2) \\  
      & \thickapprox \arg \max_w P(Y|W)P(W)\ &(3)
\end{aligned}
$$
其中$W$表示文字序列，$Y$表示语音输入，式(1)表示在给定语音输入的情况下，找到最大可能性（最大概率）的文字序列，式(2)通过贝叶斯公式将式(1)展开，$P(Y)$表示输入的语音出现的概率，与要预测的文字序列无关，可以忽略，从而得到式(3)，在式(3)中，$P(Y|W)$表示给定一个文字序列，出现该条语音的概率，即所说的声学模型；第二部分表示出现该文字序列的概率，即语音识别中所说的语音模型。接下来分别介绍声学模型和语言模型。

### 1.1 AM(Acoustic Model)
声学模型可以理解为是对发声的建模，它能够把语音输入转换成声学表示的输出，更准确的说是给出语音属于某个声学符号的概率。中文中，这个声学符号通常指的是拼音。  
在声学模型$P(Y|W)$中，可以进一步通过发音单位的序列展开，
$$
\begin{aligned}
P(Y|W) &= \arg \max_w \sum_Q P(Y,Q|W)\ &(1)\\
       & \thickapprox \arg \max_w \max_Q P(Y,Q|W)\ &(2)\\
       & \thickapprox \arg \max_w \max_Q P(Y|Q) P(Q|W)\ &(3)
\end{aligned}
$$
其中$Q$表示发音单位的序列，即中文中的拼音，专业词汇称之为音素；式(1)在已知文字序列的情况下，通过发音序列$Q$将其展开，简记为式(2)，最后近似得到式(3)。   
可以看到声学模型转化为了一个语音到发音序列的模型和发音序列到输出文字序列的字典。

#### 1.1.1 语音数据预处理
整体来看，语音数据预处理是将语音数据转换为一个特征矩阵。   

1) VAD   
语音数据首先需要把首尾端的静音切除，减少干扰，这个操作称之为VAD(Voice Activity Detection)。

2) 分帧   
接来下需要对语音进行分帧，也就将语音切分为很多段，每一小段称之为帧，目前分帧操作往往不是简单的切分（损失很多信息），而是使用移动窗口函数来实现，帧与帧之间是有重叠部分，如下图所示， 
![](../../pics/split_frame.jpg)

图中，每帧的长度是25ms，每两帧之间有25-10=15ms的重叠，即以帧长25ms帧移10ms进行分帧。   

3) 特征提取   
特征提取部分指将帧转换为向量。帧在时域上的描述能力较弱，因此需要将其做变换，变换的方式目前我知道的有MFCC、LPCC和FBANK，具体每一种提取方式后期补充。假设一条语音数据分帧后共有N段，每一帧提取特征后的向量维度为M，那么该条语音转换为N×M的矩阵，接下里则需要对该矩阵进行建模，将其转换为文字序列。 


以下介绍声学模型，
在英语中，一种常用的音素集是卡内基梅隆大学的一套由39个音素构成的音素集；汉语中，一般直接用全部声母和韵母作为音素集，另外汉语识别还分声调。

#### 1.1.2 GMM-HMM 
在GMM-HMM中，状态为比音素更细致的语音单位，通常把一个音素划分成3个状态。整体GMM-HMM的工作就是将帧识别为状态，然后将状态组合为音素。

- GMM   
GMM模型简单理解是几个高斯模型的叠加，它的主要作用是估计某个音素出现的概率。相当于HMM中观测概率。
![](../../pics/gmm.jpeg)
$$
p(x) = \sum_{j=1}^{P}P(j)p(x|j)=\sum_{j=1}^{P}P(j)N_j(x;\mu_j,\sigma_j^2)
$$

假设已知相应的GMM和HMM模型参数，
![](../../pics/gmm_pred1.jpeg)
则在上图中，给定观测序列(o1, o2, o3)的情况下，状态序列的(1, 2, 2)出现的概率可以求得。

- HMM   
在GMM求得相应的音素概率时，接下来则由HMM模型构建序列模型，HMM模型包含三部分模型参数，初始状态概率$\pi$，状态转移概率矩阵$A$和观测概率矩阵$B$，观测概率由GMM模型给出，那么只需求得$\pi$和$A$即得HMM模型。 最终通过viterbi算法求得最大概率的音素序列。

- GMM-HMM框架   
![](../../pics/gmm-hmm.jpeg)
其中GMM和HMM算法的训练均是通过EM算法

#### 1.1.3 DNN-HMM
相比较GMM-HMM模型，DNN-HMM模型改进是将GMM模型替换为DNN模型，即通过DNN模型求音素的观察概率。
DNN-HMM的模型架构图如下， 
![](../../pics/dnn-hmm.jpg)

- DNN-HMM和GMM-HMM的比较
    - 相同点   
    两者均是拟合一个观测序列的概率分布，然后将其输出作为HMM模型的观测概率矩阵B
    - 不同
        - DNN建模时不需要假设数据的分布，而GMM则需要假设数据服从高斯分布
        - GMM属于生成模型，直接生成似然概率，即观察概率，DNN属于判别模型，有监督地进行学习，生成后验概率 
        - 训练GMM模型输入的是单帧特征，模型DNN模型输入是多帧，DNN建模具有较强的上下文信息，且引入非线性拟合能力，整理效果要优于GMM   
> DNN输入的窗长（拼接的帧数）是事前固定的，因此DNN的结果只能学习到固定的输入到输入的映射关系，对时序信息更长时相关性的建模灵活性不足

![](../../pics/dnn-hmm_gmm-hmm_compare.jpg)

后续基于NN-HMM框架，相继有RNN-HMM，LSTM-HMM，CNN-HMM以及CLDNN(cnn-lstm-dnn hybrid)等模型，这些模型主要生成观测概率，之后利用HMM来生成预测的音素。

**CTC**

#### 1.1.4 LSTM-CTC
BLSTM-RNN
在LSTM和BLSTM之间折中，
LC-LSTM（2015）和CSC-LSTM，improved LC-LSTM（alibaba，2017.1）

deep rnn model

Highway-LSTM & LC-LSTM(2015) Residual-LSTM(2017.1)

FSMN（2015）
LFR(2016)

Jasper(2019)



#### 1.1.5 DFCNN（2016）

### 1.2 LM

#### 1.2.1 N-gram

#### 1.2.2 基于概率图的最大熵隐马尔可夫模型

#### 1.2.3 transformer(2017)
transformer XL
#### 1.2.4 CBHG
#### WFST

#### 1.2.2 
## 2. end2end

## 3. Package&API